In [1]:
import pandas as pd 
import numpy as np 
import re
from openai import OpenAI
import ast
from tqdm import tqdm 
import matplotlib.pyplot as plt 
import plotly.express as px
import statsmodels.formula.api as smf
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import spearmanr, pearsonr
import pickle
import itertools


In [2]:
client = OpenAI(api_key=open("/Users/hi387/Desktop/open_ai_key.txt", "r").read().strip())

In [28]:
df = pd.read_csv("../data/102_tasks_with_sources_clean.csv")
df.head()

,Task Name,Source,Link,Stimulus Complex,Goal Directives
0,9 Dot Problem,Maier 1930,https://doi.org/10.1037/h0073232,Participants are given a 3x3 grid of nine dots...,"Within a limited amount of time, participants ..."
1,Abstract grid task,Adams et al. 2021,https://doi.org/10.1038/s41586-021-03380-y,"Participants are given a shared grid, in which...",The goal is to make the grid appear exactly sy...
2,Advertisement writing,Whiting et al. 2019,https://dl.acm.org/doi/pdf/10.1145/3359311,Participants were given the description of a p...,The goal was to write an online text advertise...
3,Aerospace Investment (Role-playing),Sanpietro 2019,https://www.pon.harvard.edu/daily/teaching-neg...,Participants are randomly paired for a scored ...,Participants are explicitly instructed that th...
4,Allocating resources to programs,Whiting et al. 2019,https://dl.acm.org/doi/pdf/10.1145/3359311,Participants are given a list of complex compe...,The goal is to collectively decide how to allo...


In [43]:
def get_task_string(task):
    return f"""{task['Task Name']}
    Setup: {task['Stimulus Complex']}
    Goal : {task['Goal Directives']}
    """  

    

def task_description_to_dimensions(ref_task_list):
    system_prompt = f"You will be presented with a description of {len(ref_task_list)} different tasks." + """ After looking at all of the task descriptions, determine the set of dimensions along which the tasks vary. For each of these dimensions, provide a short description of the dimension, along with how it can be operationalized. As an example, if the dimension is 'Time-solvability', the question can be 'Is a participant able to come up with a provably correct solution, assuming sufficiewnt ability, time, motivation, and resources? After providing the different dimensions that best describe these tasks, provide an overall rationale for your choice of these tasks as well. 
    
        YOUR ANSWER MUST FOLLOW THIS JSON FORMAT : {"dimensions": [{[DIMENSION 1 NAME] : [DIMENSION 1 DESCRIPTION]}, {[DIMENSION 2 NAME] : [DIMENSION 2 DESCRIPTION], ...], 'rationale' : [YOUR RATIONALE]}"""

    prompts = []
    
    for i, row in ref_task_list.iterrows():
        prompts.append(f"Task {i+1} is: {get_task_string(row)}")
        
    messages = [{"role": "system", "content": system_prompt}]
    
    for prompt in prompts:
        messages.append({"role": "user", "content": prompt})
    
    response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=messages,
        temperature=0)
    
    return response

In [138]:
iteration = 3

In [160]:
num_samples = 50
sample = df.sample(n = num_samples).reset_index()
sample

,index,Task Name,Source,Link,Stimulus Complex,Goal Directives
0,42,Iterated Snowdrift Game (Without Punishment),"Jiang, Perc, and Szolnoki 2013",https://doi.org/10.1371/journal.pone.0064677,Participants are randomly paired in a group ac...,Participants are instructed that their goal is...
1,24,Estimating Factual Quantities,"Silver, Mellers, and Tetlock 2021",https://doi.org/10.1016/j.jesp.2021.104157,Participants are given a series of questions o...,The goal is to answer as many questions from t...
2,31,Game of Clue - Terrorist Attack,Shore et al. 2015,https://doi.org/10.1287/orsc.2015.0980,Participants work in a shared web interface in...,The goal was to solve for four facts:\n• Who w...
3,91,Visual Oddball Target,Hertz et al. 2016,https://dx.doi.org/10.1037/xhp0000145,Participants see several groupings of visual s...,The goal is to visually identify when there is...
4,76,Shopping plan,Wooley et al. 2010,https://doi.org/10.1126/science.1193147,Each participant has a grocery list and a map ...,Participants have to plan a shopping trip as t...
5,55,Organization Game,Krackhardt and Steele 1988,https://doi.org/10.2307/2786835,Participants are randomly assigned to one of f...,The goal is to maximize the organizational eff...
6,3,Aerospace Investment (Role-playing),Sanpietro 2019,https://www.pon.harvard.edu/daily/teaching-neg...,Participants are randomly paired for a scored ...,Participants are explicitly instructed that th...
7,6,Architectural design task,Wooley et al. 2010,https://doi.org/10.1126/science.1193147,Participants receive written and video instruc...,"The goal is to design a house, a garage, and a..."
8,37,Image rating,Engel et al. 2014,https://doi.org/10.1371/journal.pone.0115212,Participants see pictures of a series of produ...,The goal is to give a rating that is as close ...
9,99,Word completion given starting letter,Wooley et al. 2010,https://doi.org/10.1126/science.1193147,Participants are given a starting letter and e...,The goal is to generate a list of valid Englis...


In [161]:
list(sample['index'])

[42,
 24,
 31,
 91,
 76,
 55,
 3,
 6,
 37,
 99,
 47,
 66,
 75,
 79,
 34,
 38,
 19,
 64,
 33,
 12,
 97,
 65,
 56,
 11,
 61,
 49,
 39,
 30,
 25,
 96,
 74,
 26,
 62,
 54,
 40,
 17,
 69,
 70,
 85,
 22,
 43,
 60,
 10,
 14,
 84,
 48,
 46,
 100,
 90,
 82]

In [162]:
response = task_description_to_dimensions(sample)
response

ChatCompletion(id='chatcmpl-8hfsENzUleuqnPp7yTaCmh9briD2v', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{\n  "dimensions": [\n    {\n      "Complexity": "The level of difficulty involved in completing the task, which can range from simple (e.g., recalling a list of words) to complex (e.g., negotiating a business deal). This can be operationalized by the number of rules, steps, or variables that must be considered to perform the task."\n    },\n    {\n      "Collaboration": "The extent to which the task requires interaction and cooperation with others. This can range from individual tasks to tasks requiring coordination with multiple participants. It can be operationalized by the number of participants involved and the degree of communication required."\n    },\n    {\n      "Cognitive Load": "The amount of mental effort and working memory usage required by the task. This can be operationalized by assessing the amount of 

In [163]:
response.choices[0].message.content.replace('\n', '')[7:-3]

'{  "dimensions": [    {      "Complexity": "The level of difficulty involved in completing the task, which can range from simple (e.g., recalling a list of words) to complex (e.g., negotiating a business deal). This can be operationalized by the number of rules, steps, or variables that must be considered to perform the task."    },    {      "Collaboration": "The extent to which the task requires interaction and cooperation with others. This can range from individual tasks to tasks requiring coordination with multiple participants. It can be operationalized by the number of participants involved and the degree of communication required."    },    {      "Cognitive Load": "The amount of mental effort and working memory usage required by the task. This can be operationalized by assessing the amount of information that must be retained and manipulated to complete the task."    },    {      "Time Pressure": "The degree to which a task is constrained by time, which can affect decision-mak

In [164]:
response_dict = ast.literal_eval(response.choices[0].message.content.replace('\n', '')[7:-3])

In [165]:
response_dict['tasks'] = list(sample['index'])

In [166]:
import json
path = '../data/task_to_dimension_responses/' + str(num_samples) + '_tasks_' + str(iteration) + '.json'
with open(path, 'w') as f:
    json.dump(response_dict, f, indent = 4)